1. Key Entities & Relationships
Teams: Each team belongs to a grade and has constraints on when they can or cannot play.
Club: Teams belong to a club. There may be constraints on when a whole club may want to play.
Grades: A grouping of teams where teams play each other in a round-robin format.
Games: Scheduled matchups between teams at a specific time and date.
Fields: The fields that can be used for match ups.
Week: A roster of all games to be played in that week.
Roster: A complete schedule that assigns teams to timeslots while respecting constraints.
Timeslots: A complete calendar of timeslots that are available for use in the roster.

2. Constraints & Optimization Goals
We need to ensure:

Hard Constraints (Must Be Enforced)

Teams must not be scheduled to play at times they explicitly cannot play.
No team is scheduled for two games at the same time.
Games must be assigned to valid time slots.
Teams must only play other teams in the same grade. 
A team should not have two byes in a row. 
Each team appears once per round (or gets a bye).


Soft Constraints (Optimization Goals)

Teams should be scheduled to play at their preferred times when possible.
Balanced matchups: Teams should play each other an even number of times over the competition.
Even distribution of byes (if a grade has an odd number of teams).


Objective Function:
Maximize the fulfillment of preferences while minimizing schedule conflicts.

Constraint Types (Potential Constraints to be enforced):

A club may want its games in afternoons. Thus, we need to be able to apply a constraint to an entire club.
A club may want to have all of its games on one field for a particular week, called a Club Day. Furthermore, these games should be contiguous, with only the top or tail breaking the constraint.

A team may not be able to play a specific time, or a range of times, on a particular day. Thus, we must be able to use =, <, >, to determine when a team can play on a given day.
A team may not be able to play a specific time, or a range of times in perpetuity, so we need to be able to add a continuing time restraint.



In [38]:
from pydantic import BaseModel, Field
from typing import List, Dict, Tuple
from datetime import time as tm
from datetime import timedelta, datetime
from ortools.sat.python import cp_model

class Timeslot(BaseModel):
    day: str = Field(..., description="Day of the game (e.g., 'Saturday', 'Sunday')")
    time: tm = Field(..., description="Time of the game (e.g., 14:00 for 2 PM)")
    week: int = Field(..., description="The week number for the season")
    
class Team(BaseModel):
    name: str = Field(..., description="Name of the team")
    club: str = Field(..., description="Club the team belongs to")
    grade: str = Field(..., description="Grade the team belongs to")
    preferred_times: List[Timeslot] = Field(default=[], description="Times the team prefers to play")
    unavailable_times: List[Timeslot] = Field(default=[], description="Times the team cannot play")


class PlayingField(BaseModel):
    name: str = Field(..., description="Name of the field")
    location: str = Field(..., description="Location of the field")

class Grade(BaseModel):
    name: str = Field(..., description="Grade name")
    teams: List[str] = Field(..., description="List of team names in this grade")

class Game(BaseModel):
    team1: str = Field(..., description="First team playing")
    team2: str = Field(..., description="Second team playing")
    timeslot: Timeslot = Field(..., description="Scheduled time for the game")
    field: str = Field(..., description="Field where the game is played")
    grade: str = Field(..., description="Grade the game belongs to")

class WeeklyDraw(BaseModel):
    week: int = Field(..., description="Week number in the season")
    games: List[Game] = Field(..., description="Games scheduled for this week")
    
class Club(BaseModel):
    name: str = Field(..., description="Club name")
    teams: List[str] = Field(..., description="List of team names belonging to this club")
    preferred_times: List[Timeslot] = Field(default=[], description="Preferred play times for the club")
    constraints: List[str] = Field(default=[], description="Special scheduling constraints for the club")

class Roster(BaseModel):
    weeks: List[WeeklyDraw] = Field(..., description="Complete schedule for the season")





In [48]:
from abc import ABC, abstractmethod
from ortools.sat.python import cp_model

class Constraint(ABC):
    """Abstract base class for all scheduling constraints."""
    
    @abstractmethod
    def apply(self, model: cp_model.CpModel, X: dict, data: dict):
        """Apply constraint to the OR-Tools model."""
        pass

# Hard constraints
class NoDoubleBookingConstraint(Constraint):
    """Ensure no team is scheduled for two games at the same time."""
    
    def apply(self, model, X, data):
        teams = data['teams']
        timeslots = data['timeslots']
        fields = data['fields']
        games = data['games']
        
        for team in teams:
            for t in timeslots:
                scheduled_games = [X[(t1, t2, t.day, t.time, t.week, f.name)] for (t1, t2) in games
                                  if t1 == team.name or t2 == team.name
                                  for f in fields if (t1, t2, t.day, t.time, t.week, f.name) in X]
                model.Add(sum(scheduled_games) <= 1)  # At most one game per team per weekly draw

class OneGamePerWeekend(Constraint):
    """Ensure each team has a maximum of one game per weekend."""
    
    def apply(self, model, X, data):
        teams = data['teams']
        timeslots = data['timeslots']
        fields = data['fields']
        games = data['games']
        
        for team in teams:
            for t in timeslots:
                scheduled_games = [X[(t1, t2, t.day, t.time, t.week, f.name)] for (t1, t2) in games
                                  if t1 == team.name or t2 == team.name
                                  for f in fields if (t1, t2, t.day, t.time, t.week, f.name) in X]
                model.Add(sum(scheduled_games) <= 1)  # At most one game per team per time

class GradeMatchingConstraint(Constraint):
    """Ensure teams only play against teams in the same grade."""
    
    def apply(self, model, X, data):
        grades = data['grades']
        games = data['games']
        
        for (t1, t2) in games:
            grade1 = next(g for g in grades if t1 in g.teams)
            grade2 = next(g for g in grades if t2 in g.teams)
            if grade1 != grade2:
                for t in data['timeslots']:
                    for f in data['fields']:
                        model.Add(X[(t1, t2, t, f)] == 0)  # Disallow cross-grade games

class UnavailableTimesConstraint(Constraint):
    """Ensure teams do not play at times they cannot play."""
    
    def apply(self, model, X, data):
        teams = data['teams']
        
        for team in teams:
            for t in team.unavailable_times:
                for (t1, t2) in data['games']:
                    if t1 == team.name or t2 == team.name:
                        for f in data['fields']:
                            if (t1, t2, t.day, t.time, t.week, f.name) in X:
                                model.Add(X[(t1, t2, t.day, t.time, t.week, f.name)] == 0)

class ConsecutiveByesConstraint(Constraint):
    """Ensure a team does not have two consecutive byes."""
    
    def apply(self, model, X, data):
        teams = data['teams']
        weeks = data['weeks']
        
        for team in teams:
            for w in range(1, len(range(weeks)) - 1):
                bye1 = sum(X[(t1, t2, t.day, t.time, t.week, f.name)] for (t1, t2) in data['games']
                           if (t1 == team.name or t2 == team.name)
                           for t in data['timeslots'] for f in data['fields']) == 0
                bye2 = sum(X[(t1, t2, t.day, t.time, t.week, f.name)] for (t1, t2) in data['games']
                           if (t1 == team.name or t2 == team.name)
                           for t in data['timeslots'] for f in data['fields']) == 0
                model.Add(bye1 + bye2 <= 1)  # No two consecutive byes

# Modify create_schedule to take constraints

def create_schedule(teams, grades, fields, timeslots, num_weeks, constraints):
    model = cp_model.CpModel()
    
    games = {(t1, t2): (t1, t2) for grade in grades for t1 in grade.teams for t2 in grade.teams if t1 < t2}
    X = {(t1, t2, t, f): model.NewBoolVar(f'X_{t1}_{t2}_{t.day}_{t.time}_{f.name}')
         for (t1, t2) in games for t in timeslots for f in fields}
    
    # Apply constraints dynamically
    data = {'teams': teams, 'grades': grades, 'fields': fields, 'timeslots': timeslots, 'games': games, 'weeks': num_weeks}
    for constraint in constraints:
        constraint.apply(model, X, data)
    
    # Objective function (maximize preferred times)
    model.Maximize(sum(X[(t1, t2, t, f)] for (t1, t2) in games
                        for t in timeslots for f in fields
                        if any(t in team.preferred_times for team in teams if team.name in (t1, t2))))
    
    solver = cp_model.CpSolver()
    solver.Solve(model)
    
    return X  # Return scheduled matches


In [51]:
def generate_timeslots(start_date, end_date, days, times):
    """Generate weekly timeslots between two dates."""
    timeslots = []
    current_date = start_date
    week_number = 1
    while current_date <= end_date:
        if current_date.strftime('%A') in days:
            for t in times:
                timeslots.append({'day': current_date.strftime('%A'), 'time': t, 'week': week_number})
        if current_date.strftime('%A') == 'Monday':
            week_number += 1
        current_date += timedelta(days=1)
    return timeslots

def create_schedule(teams, grades, fields, timeslots, num_weeks, constraints):
    model = cp_model.CpModel()
    
    # Generate game pairs
    games = {(t1, t2): (t1, t2) for grade in grades for t1 in grade.teams for t2 in grade.teams if t1 < t2}
    print(f"Generated {len(games)} games.")  # Debugging

    # Generate variables
    X = {(t1, t2, t.day, t.time, t.week, f.name): model.NewBoolVar(f'X_{t1}_{t2}_{t.day}_{t.time}_{t.week}_{f.name}')
         for (t1, t2) in games for t in timeslots for f in fields}
    print(f"Generated {len(X)} decision variables.")  # Debugging
    
    # Apply constraints dynamically
    data = {'teams': teams, 'grades': grades, 'fields': fields, 'timeslots': timeslots, 'games': games, 'weeks': num_weeks}
    for constraint in constraints:
        constraint.apply(model, X, data)
    
    # Objective function (maximize preferred times)
    model.Maximize(sum(X[(t1, t2, t.day, t.time, t.week, f.name)] for (t1, t2) in games
                        for t in timeslots for f in fields
                        if any(t in team.preferred_times for team in teams if team.name in (t1, t2))))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    print(f"Solver status: {solver.StatusName()}")  # Check solver status

    # If feasible solution found, return scheduled matches
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        return {key: solver.Value(var) for key, var in X.items()}
    else:
        print("No feasible schedule found.")
        return {}



# Instantiate data
FIELDS = [PlayingField(name='South Field', location='South'),
          PlayingField(name='East Field', location='East'),
          PlayingField(name='West Field', location='West')]

CLUBS = ['Souths', 'Wests', 'Uni', 'Norths', 'Tigers', 'Port Stephens', 'Maitland', 'Crusaders', 'Colts', 'Cardiff']
GRADES = [Grade(name=f'Grade {i}', teams=[f'{club} Grade {i}' for club in CLUBS]) for i in range(1, 7)]
TEAMS = [Team(name=f'{club} Grade {grade}', club=club, grade=f'Grade {grade}') for club in CLUBS for grade in range(1, 7)]

# Generate timeslots
times = [tm(8, 0), tm(10, 0), tm(12, 0), tm(2, 0), tm(4, 0)]
timeslots = generate_timeslots(datetime(2025, 3, 4), datetime(2025, 10, 31), ['Saturday', 'Sunday'], [tm(10, 0), tm(14, 0)])
TIMESLOTS = [Timeslot(day=t['day'], time=t['time'], week=t['week']) for t in timeslots]
# Run optimizer
constraints = [NoDoubleBookingConstraint(), GradeMatchingConstraint(),  OneGamePerWeekend()]
X = create_schedule(TEAMS, GRADES, FIELDS, TIMESLOTS, num_weeks=20, constraints=constraints)


Generated 270 games.
Generated 110160 decision variables.
Solver status: OPTIMAL


In [58]:
for key in  X.keys():
    print(key)
    break

('Souths Grade 1', 'Wests Grade 1', 'Saturday', datetime.time(10, 0), 1, 'South Field')
